In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("spark")\
    .config('spark.driver.memory', '50g')\
    .config("hive.server2.thrift.port", 10000)\
    .config("spark.driver.maxResultSize", 0)\
    .config("spark.sql.hive.thriftServer.singleSession", True)\
    .enableHiveSupport()\
    .getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/13 13:01:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType,TimestampType
from pyspark.sql.types import DateType
from pyspark.sql.functions import *

schema = StructType([
    StructField("ID", IntegerType()),
    StructField("Speed", DoubleType()),
    StructField("TravelTime", DoubleType()),
    StructField("Status", IntegerType()),
    StructField("timedate", StringType()),
    StructField("LinkId", IntegerType()),
    StructField("LinkPoints", StringType()),
    StructField("EncodedLinkPoints", StringType()),
    StructField("EncodedPolyLineLvls", StringType()),
    StructField("Owner", StringType()),
    StructField("TranscomId", IntegerType()),
    StructField("Borough", StringType()),
    StructField("Link_Name", StringType())    
])

schema1 = "ID bigint, Speed float, TravelTime int, Status int, TimeDate string, LinkId int, LinkPoints string, EncodedLinkPoints string, EncodedPolyLineLvls string, Owner string, TranscomId int, Borough string,Link_Name string"
training = spark.read.format("csv") \
    .option("header", "true")\
    .schema(schema) \
    .load("file:/Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv")


In [4]:
cols= ("ID","Status","LinkId","LinkPoints","EncodedLinkPoints","EncodedPolyLineLvls","TranscomId", "Owner")
training = training.drop(*cols)
training.printSchema()
#training.show()

root
 |-- Speed: double (nullable = true)
 |-- TravelTime: double (nullable = true)
 |-- timedate: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Link_Name: string (nullable = true)



In [5]:
import numpy as np

from pyspark.mllib.stat import Statistics
summary = Statistics.colStats(training.toPandas())
print(summary.mean())  # a dense vector containing the mean value for each column
print(summary.variance())  # column-wise variance
print(summary.numNonzeros())  # number of nonzeros in each column


22/03/13 13:01:57 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: SPEED, TRAVEL_TIME, DATA_AS_OF, BOROUGH, LINK_NAME
 Schema: Speed, TravelTime, timedate, Borough, Link_Name
Expected: TravelTime but found: TRAVEL_TIME
CSV file: file:///Users/minoseah629/Downloads/DOT_Traffic_Speeds_NBE.csv


AttributeError: 'DataFrame' object has no attribute 'map'

In [ ]:
# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

# We can also use the multinomial family for binary classification
mlr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial")

# Fit the model
mlrModel = mlr.fit(training)

# Print the coefficients and intercepts for logistic regression with multinomial family
print("Multinomial coefficients: " + str(mlrModel.coefficientMatrix))
print("Multinomial intercepts: " + str(mlrModel.interceptVector))

In [7]:
spark.stop()